In [38]:
import numpy as np
import copy
from scipy.stats import multivariate_normal

In [39]:
import math
from scipy.linalg import sqrtm

In [40]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [41]:
# Defining Mean and Covariance for 1st component
def comp_one(size):
    cov1 = np.array([[2.0, 0.0],
                    [0.0, 2.0]])
    mean1 = np.array([2.0, 2.0])
    return np.random.multivariate_normal(mean=mean1, cov=cov1, size=size)

In [42]:
# Defining Mean and Covariance for 1st component
def comp_two(size):
    cov2 = np.array([[1.0, 0.5],
                    [0.5, 1.0]])
    mean2 = np.array([-1.0, 2.0])
    return np.random.multivariate_normal(mean=mean2, cov=cov2, size=size)

In [43]:
# Defining Mean and Covariance for 1st component
def comp_three(size):
    cov3 = np.array([[2.5, 0.75],
                    [0.75, 2.5]])
    mean3 = np.array([2.0, -2.0])
    return np.random.multivariate_normal(mean=mean3, cov=cov3, size=size)

In [44]:
# Defining Mean and Covariance for 1st component
def comp_four(size):
    cov4 = np.array([[2.0, -0.5],
                    [-0.5, 2.0]])
    mean4 = np.array([-1.0, -1.0])
    return np.random.multivariate_normal(mean=mean4, cov=cov4, size=size)

In [45]:
# Assigning different priors to each component for all datasets
# all priors need to add up to 1
p1 = 0.1
p2 = 0.4
p3 = 0.2
p4 = 0.3
alpha_true = [p1, p2, p3, p4]

In [46]:
true_param_combination = []
cov1 = np.array([[2.0, 0.0],
                [0.0, 2.0]])
mean1 = np.array([2.0, 2.0])
alpha1 = 0.1
cov2 = np.array([[1.0, 0.5],
                [0.5, 1.0]])
mean2 = np.array([-1.0, 2.0])
alpha2 = 0.4
cov3 = np.array([[2.5, 0.75],
                [0.75, 2.5]])
mean3 = np.array([2.0, -2.0])
alpha3 = 0.2
cov4 = np.array([[2.0, -0.5],
                [-0.5, 2.0]])
mean4 = np.array([-1.0, -1.0])
alpha4 = 0.3
true_param_combination.append([alpha1, mean1, cov1])
true_param_combination.append([alpha2, mean2, cov2])
true_param_combination.append([alpha3, mean3, cov3])
true_param_combination.append([alpha4, mean4, cov4])
np.array(true_param_combination)

array([[0.1, array([2., 2.]), array([[2., 0.],
       [0., 2.]])],
       [0.4, array([-1.,  2.]), array([[1. , 0.5],
       [0.5, 1. ]])],
       [0.2, array([ 2., -2.]),
        array([[2.5 , 0.75],
       [0.75, 2.5 ]])],
       [0.3, array([-1., -1.]),
        array([[ 2. , -0.5],
       [-0.5,  2. ]])]], dtype=object)

In [87]:
# Defining our dataset generator function
def dataset_generator(alpha, size):
    uniform_dist = np.random.uniform(0, 1, size)
    count = [0, 0, 0, 0]
    for data in uniform_dist:
        if data <= alpha[0]:
            count[0] += 1
        elif alpha[0] < data <= alpha[0] + alpha[1]:
            count[1] += 1
        elif alpha[0] + alpha[1] < data <= alpha[0] + alpha[1] + alpha[2]:
            count[2] += 1
        elif alpha[0] + alpha[1] + alpha[2] < data <= 1:
            count[3] += 1
    comp1_data = comp_one(count[0])
    comp2_data = comp_two(count[1])
    comp3_data = comp_three(count[2])
    comp4_data = comp_four(count[3])
    return np.array(np.vstack([comp1_data, comp2_data, comp3_data, comp4_data]))
        

In [48]:
# Dataset 1: Sample Size = 10
dataset1 = dataset_generator(alpha_true, 100)

In [49]:
def partition_data(dataset, partitions):
    partition_width = len(dataset) / partitions
    partitioned_dataset = []
    for i in range(partitions):
        partitioned_dataset.append(dataset[int(i*partition_width):int((i+1)*partition_width)])
    return partitioned_dataset

In [50]:
def is_converging(parameters):
    if len(parameters) == 1:
        return False
    parameter1 = parameters[-1]
    parameter2 = parameters[-2]
    for i in range(np.array(parameter1).shape[0]):
        if abs(parameter1[i][1][0] - parameter2[i][1][0]) < 0.01:
            return True
        elif abs(parameter1[i][1][1] - parameter2[i][1][1]) < 0.01:
            return True
    return False

In [51]:
def initialize_parameters(order):
    alpha = 0
    mean = [0, 0]
    cov = [[1, 0], [0, 1]]
    base_parameter = []
    for i in range(order):
        base_parameter.append([alpha, mean, cov])
    parameter_head = [base_parameter]
    return parameter_head

In [52]:
def randomize_training_start(parameters_list, order, dataset, N=1):
    random_list = []
    for i in range(N):
        for j in range(order):
            alpha = np.random.uniform(0, 1)
            mean = np.random.uniform(dataset.min(), dataset.max())
            check = True
            while check:
                cov = (np.random.uniform(dataset.min(), dataset.max()))**2
                check = not np.all(np.linalg.eigvalsh(np.array([[cov, 0], [0, cov]])) > 0)
            parameters_list[0][j][0] = alpha
            parameters_list[0][j][1] = [mean, mean]
            parameters_list[0][j][2] = [[cov, 0], [0, cov]]
        random_list.append(parameters_list)
    return random_list

In [64]:
def model_log_likelihood(dataset, parameters):
    parameters = np.array(parameters)
    log_likelihood = 0
    for data in dataset:
        likelihood = 0
        for param in parameters:
            print('Parameters for {}:\n alpha: {}\n Mean: {} \n Covariance: {}'.format(data, param[0], param[1], param[2]))
            likelihood += param[0]*multivariate_normal(param[1], param[2]).pdf(data)
            print('Likelihood: {}, Total Likelihood so far: {}'.format(param[0]*multivariate_normal(param[1], param[2]).pdf(data), likelihood))
        log_likelihood += np.log(likelihood)
        print('Log_Likelihood: {}'.format(log_likelihood))
    return log_likelihood

In [66]:
multivariate_normal([2, 2], [[2, 0], [0, 2]]).pdf([2, 2])

0.07957747154594767

In [65]:
true_likelihood = model_log_likelihood(dataset1, true_param_combination)
true_likelihood

Parameters for [3.5555607  3.35399828]:
 alpha: 0.1
 Mean: [2. 2.] 
 Covariance: [[2. 0.]
 [0. 2.]]
Likelihood: 0.002748006753761975, Total Likelihood so far: 0.002748006753761975
Parameters for [3.5555607  3.35399828]:
 alpha: 0.4
 Mean: [-1.  2.] 
 Covariance: [[1.  0.5]
 [0.5 1. ]]
Likelihood: 1.2965431641985973e-06, Total Likelihood so far: 0.0027493032969261736
Parameters for [3.5555607  3.35399828]:
 alpha: 0.2
 Mean: [ 2. -2.] 
 Covariance: [[2.5  0.75]
 [0.75 2.5 ]]
Likelihood: 4.3182576447118664e-05, Total Likelihood so far: 0.002792485873373292
Parameters for [3.5555607  3.35399828]:
 alpha: 0.3
 Mean: [-1. -1.] 
 Covariance: [[ 2.  -0.5]
 [-0.5  2. ]]
Likelihood: 4.4098911864046634e-08, Total Likelihood so far: 0.0027925299722851562
Log_Likelihood: -5.880807293941488
Parameters for [2.64536891 2.53206249]:
 alpha: 0.1
 Mean: [2. 2.] 
 Covariance: [[2. 0.]
 [0. 2.]]
Likelihood: 0.006680868937224193, Total Likelihood so far: 0.006680868937224193
Parameters for [2.64536891 2.53

Likelihood: 0.03953208998724098, Total Likelihood so far: 0.03969227866753545
Parameters for [-1.95241982  2.02393609]:
 alpha: 0.2
 Mean: [ 2. -2.] 
 Covariance: [[2.5  0.75]
 [0.75 2.5 ]]
Likelihood: 1.5064763678812792e-06, Total Likelihood so far: 0.03969378514390333
Parameters for [-1.95241982  2.02393609]:
 alpha: 0.3
 Mean: [-1. -1.] 
 Covariance: [[ 2.  -0.5]
 [-0.5  2. ]]
Likelihood: 0.0024810708293039295, Total Likelihood so far: 0.04217485597320726
Log_Likelihood: -105.19254482614794
Parameters for [-2.85978141  1.46185344]:
 alpha: 0.1
 Mean: [2. 2.] 
 Covariance: [[2. 0.]
 [0. 2.]]
Likelihood: 2.0188879963593126e-05, Total Likelihood so far: 2.0188879963593126e-05
Parameters for [-2.85978141  1.46185344]:
 alpha: 0.4
 Mean: [-1.  2.] 
 Covariance: [[1.  0.5]
 [0.5 1. ]]
Likelihood: 0.011772026618412517, Total Likelihood so far: 0.01179221549837611
Parameters for [-2.85978141  1.46185344]:
 alpha: 0.2
 Mean: [ 2. -2.] 
 Covariance: [[2.5  0.75]
 [0.75 2.5 ]]
Likelihood: 5.80

 [0.5 1. ]]
Likelihood: 1.2665926500787894e-10, Total Likelihood so far: 0.001580134267033389
Parameters for [ 2.9184183  -0.37129766]:
 alpha: 0.2
 Mean: [ 2. -2.] 
 Covariance: [[2.5  0.75]
 [0.75 2.5 ]]
Likelihood: 0.00753960387408913, Total Likelihood so far: 0.009119738141122519
Parameters for [ 2.9184183  -0.37129766]:
 alpha: 0.3
 Mean: [-1. -1.] 
 Covariance: [[ 2.  -0.5]
 [-0.5  2. ]]
Likelihood: 0.00026626941801398906, Total Likelihood so far: 0.009386007559136508
Log_Likelihood: -209.80538968945953
Parameters for [ 1.3907528  -1.45627177]:
 alpha: 0.1
 Mean: [2. 2.] 
 Covariance: [[2. 0.]
 [0. 2.]]
Likelihood: 0.0003660068465361748, Total Likelihood so far: 0.0003660068465361748
Parameters for [ 1.3907528  -1.45627177]:
 alpha: 0.4
 Mean: [-1.  2.] 
 Covariance: [[1.  0.5]
 [0.5 1. ]]
Likelihood: 2.293018596585483e-09, Total Likelihood so far: 0.00036600913955477137
Parameters for [ 1.3907528  -1.45627177]:
 alpha: 0.2
 Mean: [ 2. -2.] 
 Covariance: [[2.5  0.75]
 [0.75 2.5 ]

 [-0.5  2. ]]
Likelihood: 0.007718703974253447, Total Likelihood so far: 0.016883127404759238
Log_Likelihood: -313.1042238354577
Parameters for [-0.61061029 -2.25788298]:
 alpha: 0.1
 Mean: [2. 2.] 
 Covariance: [[2. 0.]
 [0. 2.]]
Likelihood: 1.5575323517985214e-05, Total Likelihood so far: 1.5575323517985214e-05
Parameters for [-0.61061029 -2.25788298]:
 alpha: 0.4
 Mean: [-1.  2.] 
 Covariance: [[1.  0.5]
 [0.5 1. ]]
Likelihood: 1.2398500759910988e-07, Total Likelihood so far: 1.5699308525584322e-05
Parameters for [-0.61061029 -2.25788298]:
 alpha: 0.2
 Mean: [ 2. -2.] 
 Covariance: [[2.5  0.75]
 [0.75 2.5 ]]
Likelihood: 0.0032142739338611812, Total Likelihood so far: 0.0032299732423867657
Parameters for [-0.61061029 -2.25788298]:
 alpha: 0.3
 Mean: [-1. -1.] 
 Covariance: [[ 2.  -0.5]
 [-0.5  2. ]]
Likelihood: 0.01657612374001519, Total Likelihood so far: 0.019806096982401954
Log_Likelihood: -317.0259892957354
Parameters for [ 0.64685161 -1.85643869]:
 alpha: 0.1
 Mean: [2. 2.] 
 Co

-399.9165496199139

In [19]:
# This function will perform our E Step of the EM algorithm and accordingly provide us new list of posteriors for each sample
# for the next iteration
def e_step(parameters, dataset):
    posterior_list = []
    for (x1, x2) in dataset:
        post_param = []
        for (alpha, mean, cov) in parameters:
            post_param.append(alpha*multivariate_normal(mean, cov).pdf([x1, x2]))
        posterior_list.append(post_param)
    return posterior_list

In [20]:
# This function will perform our M Step of the EM algorithm and accordingly provide us new alpha, mean and covariance 
# for the next iteration
def m_step(posterior_list, dataset):
    newparameters = []
    posterior_list = np.array(posterior_list)
    for i in range(posterior_list.shape[1]):
        alpha_new = sum(posterior_list[:, i]) / len(posterior_list[:, i])
        mean_new = sum([np.array(dataset[j]) * posterior_list[j, i] for j in range(len(dataset))]) / sum(posterior_list[:, i])
        cov_new = sum([np.matmul(np.array(dataset[j] - mean_new).reshape(2, 1),np.array(dataset[j] - mean_new).reshape(1, 2))
                       * posterior_list[j, i] for j in range(len(dataset))]) / sum(posterior_list[:, i])
        newparameters.append([alpha_new, mean_new, cov_new])
    return newparameters

In [21]:
part_data = partition_data(dataset1, 10)
# parameters_list = initialize_parameters(order)
random_list_of_parameters_list = [true_param_combination]

In [22]:
my_param_list = copy.deepcopy([true_param_combination])
data = copy.deepcopy(part_data)
val_data = data.pop(0)
train_data = []
[train_data.extend(part) for part in data]
print("Train_data_len: {}, Val_data_len: {}".format(len(train_data), len(val_data)))
count = 0

Train_data_len: 90, Val_data_len: 10


In [37]:
posterior_list = e_step(my_param_list[-1], train_data)
posterior_listterior_list

NameError: name 'posterior_listterior_list' is not defined

In [ ]:
new_params = m_step(posterior_list, train_data)
print(np.array(new_params))

In [ ]:
for param_list in random_list_of_parameters_list:
    log_likelihood_list = []
    for i in range(k):
        my_param_list = copy.deepcopy(param_list)
        data = copy.deepcopy(part_data)
        val_data = data.pop(i)
        train_data = []
        [train_data.extend(part) for part in data]
        count = 0
#               Training on the Model
        while not is_converging(my_param_list):
            posterior_list = e_step(my_param_list[-1], train_data)
#                     print("Posterior_list: \n{}".format(posterior_list))
            new_params = m_step(posterior_list, train_data)
#                     print("New parameters after the step {} are \n{}".format(count+1, new_params))
            my_param_list.append(new_params)
            count += 1

In [ ]:
from sklearn import mixture

In [ ]:
gmm = mixture.GaussianMixture(4)

In [ ]:
gmm.fit(dataset_generator(alpha_true, 10000))

In [ ]:
gmm.means_

In [ ]:
gmm.predict(dataset1)

In [24]:
def k_fold_cross_validation(dataset, gmm_model_orders=[1, 2, 3, 4, 5, 6], k=10):
#     paramaters_list = []
#     dataset = dataset_generator(alpha, dataset_size)
    part_data = partition_data(dataset, k)
    model_log_likelihood_list = []
    model_best_param = []
    for order in gmm_model_orders:
        print('K-fold for Model order {}'.format(order))
        parameters_list = initialize_parameters(order)
        random_list_of_parameters_list = randomize_training_start(parameters_list, order, dataset, N=1)
        tot_log_likelihood_list = []
        tot_fold_iter_param = []
        for param_list in random_list_of_parameters_list:
            log_likelihood_list = []
            fold_iter_param = []
            for i in range(k):
                my_param_list = copy.deepcopy(param_list)
                data = copy.deepcopy(part_data)
                val_data = data.pop(i)
                train_data = []
                [train_data.extend(part) for part in data]
                count = 0
#               Training on the Model
                while not is_converging(my_param_list):
                    try:
                        posterior_list = e_step(my_param_list[-1], train_data)
                    except Exception as e:
                        break
#                     print("Posterior_list: \n{}".format(posterior_list))
                    new_params = m_step(posterior_list, train_data)
#                     print("New parameters after the step {} are \n{}".format(count+1, new_params))
                    my_param_list.append(new_params)
                    count += 1
                print("finished after {} iterations".format(count))
#                 print(my_param_list[-1])
#               Validating on the Model
                log_likelihood_list.append(model_log_likelihood(dataset, my_param_list[-1]))
                fold_iter_param.append(my_param_list[-1])
            tot_fold_iter_param.append(fold_iter_param)
            tot_log_likelihood_list.append(log_likelihood_list)
            max_list = [max(rand_log_lik_list) for rand_log_lik_list in tot_log_likelihood_list]
            fold_iter_param = tot_fold_iter_param[max_list.index(max(max_list))]
            log_likelihood_list = tot_log_likelihood_list[max_list.index(max(max_list))]
        model_best_param.append(fold_iter_param[log_likelihood_list.index(max(log_likelihood_list))])
        model_log_likelihood_list.append(sum([likely for likely in log_likelihood_list if not np.isinf(abs(likely))]))
    print(model_log_likelihood_list)
    print("Model having the highest log likelihood is {}".format(model_log_likelihood_list.index(max(model_log_likelihood_list)) + 1))
    return model_log_likelihood_list, model_best_param

In [36]:
model_likelihoods1, best_model_parameters1 = k_fold_cross_validation(dataset1)

K-fold for Model order 1
finished after 10 iterations


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


finished after 8 iterations
finished after 11 iterations
finished after 3 iterations
finished after 11 iterations
finished after 17 iterations
finished after 9 iterations
finished after 19 iterations
finished after 8 iterations
finished after 10 iterations
K-fold for Model order 2
finished after 10 iterations
finished after 10 iterations
finished after 10 iterations
finished after 12 iterations
finished after 9 iterations
finished after 9 iterations
finished after 9 iterations
finished after 6 iterations
finished after 7 iterations
finished after 6 iterations
K-fold for Model order 3
finished after 9 iterations
finished after 10 iterations
finished after 10 iterations
finished after 6 iterations
finished after 7 iterations
finished after 5 iterations
finished after 5 iterations
finished after 6 iterations
finished after 6 iterations
finished after 6 iterations
K-fold for Model order 4
finished after 2 iterations


LinAlgError: singular matrix

In [ ]:
pip install gnumpy

In [1]:
import gnumpy

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('gnumpy: failed to import cudamat. Using npmat instead. No GPU will be used.')? (gnumpy.py, line 52)

In [ ]:
dataset_generator(alpha_true, 10)

In [91]:
from sklearn.cluster import KMeans
order = 3
dataset = dataset_generator(alpha_true, 10)
alpha = np.zeros(order) + 1/order
k_means_model = KMeans(n_clusters=order).fit(dataset)
mean = np.array(k_means_model.cluster_centers_)
split_data = []
for i in range(order):
    split_data.append([])

In [92]:
for j in range(len(dataset)):
    print(k_means_model.labels_[j])
    split_data[k_means_model.labels_[j]].append(dataset[j].tolist())
split_data

1
1
1
1
0
2
2
2
0
0


[[[-0.19111708519275972, -3.283848813750312],
  [-0.19826546973816783, -0.8274271573151912],
  [0.07838468301428225, -2.758904462481279]],
 [[-1.348058607129082, 3.061419767888537],
  [-0.06045692925981583, 3.0814413644384837],
  [-1.3744634228948582, 1.2801553795243221],
  [-2.2627011317975843, 1.313730605945496]],
 [[2.4560164042998918, -3.499845575529944],
  [2.4576918722480285, 0.6591991761206821],
  [2.6662548968706843, -1.741400956620864]]]

In [96]:
cov = []
for i in range(order):
    cov.append(np.cov(np.array(split_data[i]).T))
    print(cov[i])

[[ 0.0248696  -0.06840492]
 [-0.06840492  1.67336308]]
[[0.82172042 0.65844451]
 [0.65844451 1.04985674]]
[[ 0.01461693 -0.02066922]
 [-0.02066922  4.35877695]]


In [79]:
a = [1 ,2, 3, 4]
b = [[1, 2], [3, 4], [5, 6], [7, 8]]
c = [[1, 2], [3, 4], [5, 6], [7, 8]]

In [80]:
np.vstack([b, c])

array([[1, 2],
       [3, 4],
       [5, 6],
       [7, 8],
       [1, 2],
       [3, 4],
       [5, 6],
       [7, 8]])

In [84]:
uniform_dist = np.random.uniform(0, 1, 10)
count = [0, 0, 0, 0]
for data in uniform_dist:
    if data <= alpha[0]:
        count[0] += 1
    elif alpha[0] < data <= alpha[0] + alpha[1]:
        count[1] += 1
    elif alpha[0] + alpha[1] < data <= alpha[0] + alpha[1] + alpha[2]:
        count[2] += 1
    elif alpha[0] + alpha[1] + alpha[2] < data <= 1:
        count[3] += 1
comp1_data = comp_one(count[0])
comp2_data = comp_two(count[1])
comp3_data = comp_three(count[2])
comp4_data = comp_four(count[3])

In [85]:
comp1_data.shape

(4, 2)

In [86]:
np.vstack([comp1_data, comp2_data, comp3_data, comp4_data])

array([[ 2.71979694,  0.42320336],
       [ 2.60762966,  4.06515042],
       [ 3.28870802,  1.71588862],
       [ 0.69761898,  5.21286606],
       [-0.48679975,  2.75584253],
       [ 1.46646433, -0.85794735],
       [ 3.70007188, -1.33738744],
       [ 3.90386851, -1.56714133],
       [ 1.77631509, -2.4453775 ],
       [ 2.80325608, -0.41579604]])